# U1 Actividad NLP

En esta actividad se ha analizado el conjunto de datos de [OpinRankDatasetWithJudgments](http://kavita-ganesan.com/entity-ranking-data/) para hallar las respuestas a las preguntas que propone el enunciado.

Primero, nos descargamos el dataset:

In [1]:
# If you haven't downloaded the dataset, then uncomment and run. Note unzipping is os dependent
# !curl https://github.com/kavgan/OpinRank/raw/master/OpinRankDatasetWithJudgments.zip -o "data/OpinRankDatasetWithJudgments.zip"
# !7z x -y "data/OpinRankDatasetWithJudgments.zip" -odata # 7zip / windows
# !unzip "data/OpinRankDatasetWithJudgments.zip" # mac

In [2]:
from ipyparallel import Client
rc = Client()

## Lectura de datos

Dentro del zip que hemos descargados encontramos la siguiente estructura de ficheros:

- data (dir)
  - cars (dir)
  - hotels (dir)
    - data (dir)
      - city-name (dir)
        - hotel-name (file)
          - *reviews here*
      - city-name.csv (file)
        - *hotels information*
    - judgments
      - city-name

Ignoramos los datos de coches, directorio `cars`, y nos centraremos en el directorio `hotels`. Dentro de `hotels` encontramos dos directorios, `data` y `judgments`. 

En el primer directorio, data, encontramos los datos sin procesar. Estos datos se agrupan por ciudades. Cada ciudad tiene un fichero `.csv` y un directorio con su nombre. En el csv encontramos la lista de hoteles de esa ciudad y datos asociados a estos. Entre estos datos destaca el atributo `doc_id`, que contiene el nombre del fichero donde se encuentran las review de los usuarios. El fichero de las review se encuentra dentro del directorio con el nombre de la ciudad a la que pertenece.

El `judgments` encontramos un directorio por cada ciudad. Dentro de cada directorio encontramos los resultados de las puntuaciones de relevancia.

Para información más detallada podéis consultar la documentación del dataset, `OpinRankDatasetWithJudgments.pdf`.

In [3]:
import os
import functools
import numpy as np
import pandas as pd

hotels = pd.DataFrame()

# Podemos iterar y unir todos los csv porque sabemos que solo hay 10 ciudades.
# Alternativamente, usariamos una función para cargar cada ciudad de forma independiente 
# como se ha hecho con las review de cada hotel.
for name in os.listdir("data/hotels/data"):
    if name.endswith(".csv"):
      hotelTmp = pd.read_csv(f"data/hotels/data/{name}", delimiter=',', index_col=False)
      hotels = pd.concat([hotels, hotelTmp], axis=0)

# Guardamos todos los nombres de las ciudades
cities = hotels.city.unique()

hotels.head()

,doc_id,hotel_name,hotel_url,street,city,state,country,zip,class,price,num_reviews,CLEANLINESS,ROOM,SERVICE,LOCATION,VALUE,COMFORT,overall_ratingsource
0,china_beijing_holiday_inn_central_plaza,holiday inn central plaza,http://www.tripadvisor.com/ShowUserReviews-g29...,no.1 caiyuan street xuanwu district,beijing,-1,China,100053,-1,-1,247,4.786408,4.631068,4.733010,3.553398,4.699029,0.0,4.480583
1,china_beijing_hilton_beijing_wangfujing,hilton beijing wangfujing,http://www.tripadvisor.com/ShowUserReviews-g29...,no.8 wangfujing east street dongcheng,beijing,-1,China,100006,-1,-1,74,4.810345,4.844828,4.758621,4.827586,4.517241,0.0,4.751724
2,china_beijing_hotel_g,hotel g,http://www.tripadvisor.com/ShowUserReviews-g29...,a7 worker's stadium chaoyang district,beijing,-1,China,100020,-1,-1,110,4.769231,4.750000,4.576923,4.375000,4.653846,0.0,4.625000
3,china_beijing_the_regent_beijing,the regent beijing,http://www.tripadvisor.com/ShowUserReviews-g29...,no.99 jinbao street dongcheng district,beijing,-1,China,100005,-1,-1,111,4.625000,4.812500,4.437500,4.645833,4.531250,0.0,4.610417
4,china_beijing_the_st_regis_beijing,the st regis beijing,http://www.tripadvisor.com/ShowUserReviews-g29...,no.21jianguomenwai street chaoyang district,beijing,-1,China,100020,-1,-1,89,4.846154,4.646154,4.615385,4.492308,4.184615,0.0,4.556923


Como hay muchas reviews vamos a cargar el listado de reviews de cada hotel a demanda.

In [10]:
%%px
# from textblob import TextBlob
# def spellcheck(inputText):
#   blob = TextBlob(str(inputText))
#   return str(blob.correct())

# Dada una fila/row del dataframe hotels del apartado anterior, nos devuelve un dataframe con las reviews
def getReviews(hotel):
  docId = hotel["doc_id"]
  city = hotel["city"]
  fileName = f"data/hotels/data/{city}/{docId}"
  try:
    reviews = pd.read_csv(fileName, delimiter="\t", index_col=False, header=None, encoding="iso-8859-1", on_bad_lines='skip')
    reviews.columns = ["date", "title", "review", "empty"]

    # la columna de date no la vamos a usar y la ultima columna, empty, es resultado de tener un tabulador
    # a final de cada linea. No hay datos. Por lo tanto las quitamos
    reviews = reviews.drop(columns=["date", "empty"]) 

    # corregir ortografía. Es muuuuy lento
    # reviews["title"] = reviews["title"].apply(spellcheck)
    # reviews["review"] = reviews["review"].apply(spellcheck)

    return reviews
  except FileNotFoundError:
    return pd.DataFrame()
  except Exception as e:
    print(e)
    return pd.DataFrame()

In [13]:
%%px --targets 0
# test the function
reviews = getReviews(hotels.iloc[0])
print(f"cantidad de reviews: {len(reviews)}")
reviews.head()

[stdout:0] cantidad de reviews: 247


,title,review
0,Absolutely fantastic. The only negative is not...,Just about everything about this hotel is fant...
1,BEST PLACE FOR TRAVELERS,it was so good to 2 nights when i was at beiji...
2,Excellent and travel advice,"I think this Hotel is very nice, I do not see ..."
3,good place to stay,"just back from Beijing we stayed 4 nights, hot..."
4,Hidden Gem,After reading many positive reviews on Tripadv...


En el proceso de carga de las reviews vimos que los ficheros que contienen las review:
- Tienen una review por fila.
- El separador es un tabulador.
- El encode del fichero es `iso-8859-1`. Se puede abrir en `utf-8` y es legible excepto acentos y algunos caracteres especiales.

En el ejemplo podemos comprobar que la columna `date` no siempre va a tener valor.

Actualización:
  - Se ha modificado el algoritmo para que incluya soporte para corregir typos y se ha quitado la columna `date` ya que no aporta valor.
  - Se ha comentado el corrector. Se tarda en mi maquina unos 10minutos para obtener 250 reviews. Sin el corrector son 0.6s.

## 1. ¿Qué partes de una habitación son las más mencionadas?

In this section, I'm going to use wordnet of each room to look for matches. First, I'm going to look for the definition of room that we need. Then use it's hyponyms to search in the reviews.

In [ ]:
%%px
# load dependencies
import re
import tqdm
# import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# nltk.download("punkt")
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pprint

lemmatizer = WordNetLemmatizer()

In [ ]:
# check the room definition that we need
for sense in wordnet.synsets('room'):
    print(sense)
    print(sense.definition())
    print(sense.examples())
    print("-"*10)

# Synset('room.n.01') is the definition that I'm looking for

In [ ]:
room = wordnet.synset('room.n.01')
room.hyponyms()

# now we need to "correct" the spelling of user words. Ex: some reviews wrote "bathrom" instate of "bathroom". 


# https://subscription.packtpub.com/book/application-development/9781782167853/1/ch01lvl1sec16/calculating-wordnet-synset-similarity

Estas funciones nos servirán para lanzar y recuperar los procesos en paralelo

In [36]:
import functools

def concatCounter(dict1, dict2):
  for key in dict1.keys():
    if key in dict2:
      dict2[key] += dict1[key]
    else:
      dict2[key] = dict1[key]
  return dict2

def parallelize_dataframe(df, func):
  # df_split = np.array_split(df, os.cpu_count())
  # get a view on the cluster
  view = rc.load_balanced_view()
  # submit the tasks
  asyncresult = view.map_async(func, df)
  # wait interactively for results
  asyncresult.wait_interactive()
  # retrieve actual results
  result = functools.reduce(concatCounter,asyncresult.get())
  return result

In [ ]:
%%px 
def addOne(dict, key):
  if key in dict:
    dict[key] += 1
  else:
    dict[key] = 1
  return dict

def getNouns(review):
  nouns = []
  cleanText = wordCleaner(review)
  tokens = list(set(word_tokenize(cleanText)))
  taggedTokens = pos_tag(tokens, tagset='universal')
  for taggedToken in taggedTokens:
    (word, tokenType) = taggedToken
    if tokenType == "NOUN":
      nouns.append(word)
  return nouns

def wordCleaner(text):
  return re.sub("(\W(?!(\w)))+", " ", str(text)).lower()

def getSynsetsNames(synsets):
  return list(map(lambda synset: synset.name(), synsets))

rooms = getSynsetsNames(wordnet.synset('room.n.01').hyponyms()) # get room names as "constant", so we don't recalculate every time

In [ ]:
%%px
def processReview(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      roomType = getRoomType(word)
      if roomType != None:
        counter = addOne(counter, roomType)
    except Exception as e:
      print(f"failed with word '{word}' with type '{roomType}' and error code: {e}")
  return counter

def getRoomType(word):
  wordSynsets = wordnet.synsets(str(word))
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in rooms:
      return wordSynset
  return None

In [ ]:
def findAllRoomMentions(hotels):
  counter = {}
  for index, hotel in hotels.iterrows():
    reviews = getReviews(hotel)
    for reviewIdx, review in reviews.iterrows():
      counter = processReview(counter, review)
  return counter

In [35]:
# test
parallelize_dataframe(hotels[0:12], findAllRoomMentions)

findAllRoomMentions:   0%|          | 0/12 [00:00<?, ?tasks/s]

KeyboardInterrupt: 

In [22]:
import pprint
stats = {}

for city in cities:
  stats[city] =  parallelize_dataframe(hotels[hotels["city"] == city], findAllRoomMentions)

pprint.pprint(stats)

findAllRoomMentions:   0%|          | 0/12 [00:00<?, ?tasks/s]

{'beijing': {'anteroom.n.01': 608,
             'ballroom.n.01': 6,
             'barroom.n.01': 460,
             'bathroom.n.01': 1174,
             'bedroom.n.01': 215,
             'cell.n.06': 36,
             'chamber.n.03': 1,
             'classroom.n.01': 1,
             'closet.n.04': 46,
             'court.n.02': 95,
             'cubby.n.01': 3,
             'den.n.04': 1,
             'door.n.05': 439,
             'engineering.n.03': 6,
             'floor.n.10': 931,
             'gallery.n.03': 12,
             'kitchen.n.01': 85,
             'library.n.01': 4,
             'living_room.n.01': 3,
             'lounge.n.02': 323,
             'storeroom.n.01': 1,
             'study.n.05': 9,
             'sun_parlor.n.01': 3,
             'toilet.n.01': 153,
             'walk-in.n.04': 9}}


### Resultados

## 2. ¿Qué servicios pueden detectarse por cada hotel?
Ejecutamos el mismo código de búsqueda anterior mortificándolo para que esta vez busque servicios.

In [24]:
%%px --target 0
# buscar las definiciones de "que es un servicio"
hotelServices = ["restaurant", "pool", "spa", "gym", "bellman", "wifi", "television", "excursion", "clean", "hotspot"]
for service in hotelServices:
  for sense in wordnet.synsets(service):
      print(sense)
      print(sense.definition())
      print(sense.examples())
      print(sense.hyponyms())
      print(sense.hypernyms())
      print("-"*10)

[stdout:0] Synset('restaurant.n.01')
a building where people go to eat
[]
[Synset('bistro.n.01'), Synset('brasserie.n.01'), Synset('brewpub.n.01'), Synset('cafe.n.01'), Synset('cafeteria.n.01'), Synset('canteen.n.03'), Synset('canteen.n.05'), Synset('diner.n.03'), Synset('greasy_spoon.n.01'), Synset('grillroom.n.01'), Synset('hash_house.n.01'), Synset('lunchroom.n.01'), Synset('rotisserie.n.02'), Synset('steakhouse.n.01'), Synset('teashop.n.01')]
[Synset('building.n.01')]
----------
Synset('pool.n.01')
an excavation that is (usually) filled with water
[]
[Synset('cistern.n.03'), Synset('swimming_pool.n.01'), Synset('wading_pool.n.01'), Synset('water_jump.n.01')]
[Synset('excavation.n.03')]
----------
Synset('pond.n.01')
a small lake
['the pond was too small for sailing']
[Synset('fishpond.n.01'), Synset('horsepond.n.01'), Synset('mere.n.01'), Synset('millpond.n.01'), Synset('swimming_hole.n.01'), Synset('water_hole.n.01')]
[Synset('lake.n.01')]
----------
Synset('pool.n.03')
an organiz

In [25]:
%%px
services = [
  wordnet.synset('service.n.15'), # el servicio que da un camarero a los clientes
  wordnet.synset('restaurant.n.01'),
  wordnet.synset('pool.n.01'),
  wordnet.synset('watering_place.n.01'), # kinda spa
  wordnet.synset('athletic_facility.n.01'),
  wordnet.synset('baggageman.n.01'),
  wordnet.synset('bellboy.n.01'),
  wordnet.synset('lifeguard.n.01'),
  wordnet.synset('checker.n.01'),
  wordnet.synset('houseclean.v.01'),
  wordnet.synset('wireless_local_area_network.n.01'),
  wordnet.synset('television.n.01'),
  wordnet.synset('excursion.n.01'),
  wordnet.synset('gambling_house.n.01'), 
  wordnet.synset('health_spa.n.01'), 
  wordnet.synset('hotel-casino.n.01'), 
  wordnet.synset('massage_parlor.n.02'), 
  wordnet.synset('mercantile_establishment.n.01'),
  wordnet.synset('spot.n.07'),
]
serviceNames = getSynsetsNames(services)

In [27]:
%%px
from collections import OrderedDict

def removeDuplicates(list):
  return OrderedDict.fromkeys(list).keys()

def getAllHypernyms(itemName):
  allItemHypernyms = []
  for synset in wordnet.synsets(itemName):
    allItemHypernyms = allItemHypernyms + getAllHypernymsOsSynset(synset)
  return removeDuplicates(allItemHypernyms)

def getAllHypernymsOsSynset(synset):
  synsetHypernyms = synset.hypernyms()
  if len(synsetHypernyms) == 0:
    return []

  allSynsetHypernyms = []
  for hypernym in synsetHypernyms:
    allSynsetHypernyms = allSynsetHypernyms + getAllHypernymsOsSynset(hypernym)

  return synsetHypernyms + allSynsetHypernyms

In [28]:
%%px --target 0
# test
pprint.pprint(getAllHypernyms('restaurant'))

[stdout:0] odict_keys([Synset('building.n.01'), Synset('structure.n.01'), Synset('artifact.n.01'), Synset('whole.n.02'), Synset('object.n.01'), Synset('physical_entity.n.01'), Synset('entity.n.01')])


In [31]:
%%px
def lookForServices(counter, review):
  text = str(review.title) + " " + str(review.review)
  tokenizedText = getNouns(text)
  for word in tokenizedText:
    try:
      serviceType = getServiceType(word)
      if serviceType != None:
        counter = addOne(counter, word)
    except Exception as e:
      print(e)
  return counter

# En este caso en lugar de mirar si los synset de 1 nivel encajan con la lista de habitaciones posibles, vamos a hacerlo al revés.
# Vamos a coger todos los hypernyms de la palabra y ver si encaja 
def getServiceType(word):
  wordSynsets = getAllHypernyms(word)
  for wordSynset in getSynsetsNames(wordSynsets):
    if wordSynset in serviceNames:
      return wordSynset
  return None

In [32]:
def findAllServices(hotels):
  counterList = {}
  for index, hotel in tqdm.tqdm(hotels.iterrows()):
    reviews = getReviews(hotel)
    counter = {}
    for reviewIdx, review in reviews.iterrows():
      lookForServices(counter, review)
    counterList[hotel["hotel_name"]] = counter
  return counterList

# test
result = parallelize_dataframe(hotels[0:2], findAllServices)
pprint.pprint(result)

findAllServices:   0%|          | 0/12 [00:00<?, ?tasks/s]

{'hilton beijing wangfujing': {'boutique': 1,
                               'cable': 1,
                               'cables': 1,
                               'center': 3,
                               'cleaners': 1,
                               'club': 3,
                               'grill': 1,
                               'gym': 2,
                               'mall': 17,
                               'malls': 6,
                               'market': 12,
                               'markets': 2,
                               'nightclub': 1,
                               'plaza': 1,
                               'shop': 1,
                               'shops': 4,
                               'sinks': 2,
                               'stall': 3,
                               'stalls': 1,
                               'steakhouse': 1,
                               'store': 2,
                               'stores': 3,
                               'sup

In [34]:
result = parallelize_dataframe(hotels, findAllServices)
pprint.pprint(result)

findAllServices:   0%|          | 0/12 [00:00<?, ?tasks/s]

KeyboardInterrupt: 

### Resultados

Los servicios encontrados en las review de cada hotel no tiene porque ser del propio hotel.

Comentar también que el synset de 'pool.n.01' puesto para detectar si el hotel tiene o no piscina, ha terminado detectando si hay desagües (sink/s)

## 3. ¿Qué lugares y qué porcentaje de revisiones sobre cada ciudad mencionan otras ciudades, regiones o puntos de interés turístico?

En este apartado en lugar de usar nltk vamos a usar spacy. Spacy, durante la tokenización nos etiqueta el tipo de palabra que es. En nuestro caso buscaremos por las palabras que se etiqueten como `GPE`, "Geopolitical entity, i.e. countries, cities, states".

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def findGpeMentions(hotels):
  gpeMentions = {}
  for index, hotel in tqdm.tqdm(hotels.iterrows()):
    reviews = getReviews(hotel)
    for reviewIdx, review in reviews.iterrows():
      text = str(review.title) + ". " + str(review.review)
      tokens = nlp(text)
      for ent in tokens.ents:
        if ent.label_ == "GPE":
          gpeMentions = addOne(gpeMentions, ent.lower_)
      gpeMentions = addOne(gpeMentions, "reviewsAnalyzed")
  return gpeMentions

#test
pprint.pprint(findGpeMentions(hotels[10:12]))

In [ ]:
import pprint
allGpeMentions = {}

for city in cities:
  allGpeMentions[city] = findGpeMentions(hotels[hotels["city"] == city])

pprint.pprint(allGpeMentions)

### Por ejemplo, con vuestra información podríamos llegar a responder a la pregunta: ¿en Las Vegas se menciona el Gran Cañón del Colorado?
Si, para ello tendríamos que buscar en el resultado de búsqueda `allGpeMentions` por la key, `Las Vegas` y dentro de las menciones de Las Vegas buscar si existe la key `Gran Cañón del Colorado`, en el idoma de las reviews, inglés.
Hay que tener en cuenta que no se corregido el input del usuario, por lo que resultados como "Gran canyon" o "Colorado canyon" aparecerían como sitios distintos.